# LFP from single synaptic input to pyramidal cell model from Hay et al. (2011)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import LFPy
from neuron import h
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots

np.random.seed(12345)

tstop = 20
dt = 2**-4
sigma = 0.3

In [ ]:
def insert_synaptic_input(cell, synidx, syn_e=0, weight_factor=1):

    synapse_parameters = dict(
                          idx = synidx,
                          e = syn_e, # reversal potential
                          weight = 0.005 * weight_factor, # synapse weight
                          record_current = True, # record synapse current
                          syntype = 'Exp2Syn',
                          tau1 = 0.5, #Time constant, rise
                          tau2 = 2.0, #Time constant, decay
                          )
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([3.]))
    return synapse, cell


def return_electrode_grid():
    xmin, xmax = [-150, 150]
    zmin, zmax = [-50, 1000]

    dx = 100
    dz = 100
    x_grid, z_grid = np.mgrid[xmin:xmax+dx:dx, zmin:zmax+dz:dz]
    num_elecs = len(x_grid.flatten())
    elec_grid_params = dict(
                sigma = sigma,      # extracellular conductivity
                x = x_grid.flatten(),
                y = np.zeros(num_elecs) + 0,
                z = z_grid.flatten(),
                method = 'pointsource',
            )
    return elec_grid_params


In [ ]:
cell = ns.return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)

# A: Excitatory input to apical dendrite
# B: Inhibitory input to apical dendrite
# C: Excitatory input to soma

synidxs = [cell.get_closest_idx(x=0, y=0, z=600),
           cell.get_closest_idx(x=0, y=0, z=600),
           cell.get_closest_idx(x=0, y=0, z=0)]
syn_es = [0, -90, 0] 
weight_factor = [1, 4, 1]
synapse_colors = ["b", "r", "b"]

cell.__del__()

remove_list = ["Nap_Et2", "NaTa_t", "NaTs2_t", "SKv3_1",
    "SK_E2", "K_Tst", "K_Pst",
    "Im", "Ih", "CaDynamics_E2", "Ca_LVAst", "Ca", "Ca_HVA"]

elec_grid_params = return_electrode_grid()
elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
M_elec = elec.get_transformation_matrix()

v_es = []

for sim_idx in range(len(synidxs)):
    cell = ns.return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
    ns.point_axon_down(cell)   
    syn, cell = insert_synaptic_input(cell, synidxs[sim_idx], syn_es[sim_idx], weight_factor[sim_idx])
    h.dt = dt

    cell.simulate(rec_imem=True, rec_vmem=True)
    print("Max dV: ", np.max(np.abs(cell.vmem - cell.vmem[:, 0, None])))

    v_e = M_elec @ cell.imem * 1000
    v_e -= v_e[:, 0, None]
    v_es.append(v_e.copy())

    # For plotting, we still want cell
    if not sim_idx == len(synidxs) - 1:
        cell.__del__()

In [ ]:
xmin = -155
xmax = 220
zmin = -160
zmax = 1150

fig = plt.figure(figsize=[6, 5.4])
fig.subplots_adjust(bottom=0.05, top=0.98, left=0.01, right=0.99, wspace=0.2)

ax1 = fig.add_subplot(131, frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin, xmax],
                        ylim=[zmin, zmax])
ax2 = fig.add_subplot(132, frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin, xmax],
                        ylim=[zmin, zmax])
ax3 = fig.add_subplot(133, frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin, xmax],
                        ylim=[zmin, zmax])

axes = [ax1, ax2, ax3]

t1 = 20
t1_idx = np.argmin(np.abs(cell.tvec - t1))
dz = 100
dx = 100

num_elecs = len(elec.x)
max_v_e = np.max(np.abs(v_es))
lsyns = []

for sim_idx in range(len(synidxs)):

    ax = axes[sim_idx]
    zips = []
    for x, z in cell.get_pt3d_polygons():
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips, edgecolors='none',
                             facecolors='0.4', zorder=-1, rasterized=False)
    ax.add_collection(polycol)

    l_syn, = ax.plot(cell.x[synidxs[sim_idx]].mean(),
                  cell.z[synidxs[sim_idx]].mean(), c=synapse_colors[sim_idx],
                           marker='o', ls='none')
    lsyns.append(l_syn)
    
    v_e_norm = dz * 0.9 / max_v_e
    print("Max Ve: ", np.max(np.abs(v_es[sim_idx])))
    
    t_norm = cell.tvec[:t1_idx] / t1 * dx * 0.7
    for elec_idx in range(num_elecs):
        c = 'k'
        x, z = elec.x[elec_idx], elec.z[elec_idx]
        ax.plot(x, z, '.', c='k', ms=3)
        v_e_ = v_es[sim_idx][elec_idx, :t1_idx] * v_e_norm
        ax.plot(x + t_norm, z + v_e_, c=c, lw=1)

ax1.plot([150, 250], [-15, -15], c='gray', lw=1)
ax1.text(200, 0, "100 µm", ha="center", c='gray')

ax1.plot([150, 150 + t_norm[-1]], [775, 775], c='gray', lw=1)
ax1.text(180, 777, "{:d} ms".format(int(t1)), ha="center",
              va="bottom", c='gray')

ax1.plot([130, 130], [720 - max_v_e * v_e_norm, 720], c='gray', lw=1,
              clip_on=False)
ax1.text(135, 720 - 0.5 * max_v_e * v_e_norm, "{:1.2f} µV".format(max_v_e), ha="left",
              c='gray', va="center")
fig.legend([lsyns[0], lsyns[1]], ["excitatory synapse", "inhibitory synapse"], loc=(0.1, 0.01),
           frameon=False, handlelength=0.5, ncol=2)

mark_subplots([ax1, ax2, ax3], xpos=0.05, ypos=0.98)

fig.savefig("hay_single_synapse_LFP.pdf")